In [ ]:
import numpy as np

path = "energydata_Englandwales.npy"    # ← まずこれなど1つ指定
data = np.load(path, allow_pickle=True)

print("type:", type(data))
print("dtype:", getattr(data, "dtype", None))

print("\n=== raw data ===")
print(data)

# もし dict っぽければ keys を見る
if isinstance(data, dict):
    print("\nkeys:", data.keys())

# もし object 配列なら item() をチェック
if data.__class__.__name__ == "ndarray" and data.dtype == object:
    try:
        print("\nitem():", data.item())
    except:
        pass


: 